In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow.keras import utils,layers,Sequential
import tensorflow_hub as hub

In [2]:
train = utils.image_dataset_from_directory('/kaggle/input/virtual-tryon-dataset/Virtual tryon data/train',image_size=(256,256),seed=42)
test = utils.image_dataset_from_directory('/kaggle/input/virtual-tryon-dataset/Virtual tryon data/train',image_size=(256,256),seed=42)

#normalize the data
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train = train.map(process)
test = test.map(process)

Found 46588 files belonging to 5 classes.
Found 46588 files belonging to 5 classes.


In [3]:
model = Sequential([
    layers.Input(shape=(256,256,3)),
    hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/b7/classification/1'),
    #the layer which loads eff_net b7
    layers.Dense(128,activation="relu"),
    layers.Dense(64,activation='relu'),
    layers.Dense(4,activation='softmax')
])


In [4]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              66658680  
                                                                 
 dense (Dense)               (None, 128)               128128    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 4)                 260       
                                                                 
Total params: 66,795,324
Trainable params: 136,644
Non-trainable params: 66,658,680
_________________________________________________________________


In [5]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.015),
    metrics=['Accuracy']
)


In [6]:
earlystopping=tf.keras.callbacks.EarlyStopping(
    monitor='Accuracy',
    min_delta=0.0001,
    patience=2,
    restore_best_weights=True,
    start_from_epoch=1
)

tensorboard= tf.keras.callbacks.TensorBoard()
chkpt = tf.keras.callbacks.ModelCheckpoint(
    'mango.ckpt',
    monitor = 'Accuracy',
    save_best_only = True,
    mode = 'auto',
    save_freq='epoch',
)


In [7]:
with tf.device('/GPU:0'):
    model.fit(train,
              epochs=4,
              callbacks=[earlystopping,tensorboard,chkpt])

Epoch 1/4
1456/1456 [==============================] - 711s 466ms/step - loss: 0.1367 - Accuracy: 0.9545
Epoch 2/4
1456/1456 [==============================] - 631s 433ms/step - loss: 0.0515 - Accuracy: 0.9827
Epoch 3/4
1456/1456 [==============================] - 631s 433ms/step - loss: 0.0406 - Accuracy: 0.9864
Epoch 4/4
1456/1456 [==============================] - 631s 433ms/step - loss: 0.0336 - Accuracy: 0.9890


In [8]:
model.save("/kaggle/working/my_model_virtual_tryon.h5")